In [ ]:
!pip install speechbrain
!pip install torchaudio
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 20.8 MB/s eta 0:00:00


In [ ]:
import torchaudio
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# Load and preprocess the audio file
waveform, sample_rate = torchaudio.load("sample-1.mp3")
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# Convert stereo to mono if necessary
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0)
waveform = waveform.squeeze(0)

# Tokenize the audio input
input_values = processor(waveform, return_tensors="pt", sampling_rate=16000).input_values

# Perform STT
with torch.no_grad():
    logits = model(input_values).logits

# Decode the logits to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [ ]:
print(transcription[0])
import time
import psutil
import os
# Function to measure inference time
def measure_inference_time(model, input_tensor):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        start_time = time.time()
        _ = model(input_tensor)  # Perform inference
        end_time = time.time()
    return end_time - start_time

# Example: Assume input_tensor is preprocessed audio tensor
input_tensor = torch.rand(1, 16000)  # Simulate an audio input of 1-second at 16kHz

# Measure for non-quantized model
inference_time_non_quant = measure_inference_time(model, input_tensor)
print(f"Inference time (non-quantized): {inference_time_non_quant} seconds")

# Function to check RAM usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 ** 2)  # Return memory usage in MB

# Example: Measure RAM usage before and after inference for non-quantized and quantized models
print(f"Initial RAM usage: {get_memory_usage()} MB")

# Inference with non-quantized model
_ = model(input_tensor)
print(f"RAM usage after non-quantized model: {get_memory_usage()} MB")

IT IS ELEVEN VINCESSIER N IT'S INNORING IS ITS MILE WARE MAY SEND A LITTLE NAY N THE SHEN YOU WITH HER AW MEN I HA DELL DE FRAIL
Inference time (non-quantized): 0.7033300399780273 seconds
Initial RAM usage: 2582.50390625 MB
RAM usage after non-quantized model: 2582.50390625 MB


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
from peft import LoraConfig, get_peft_model
from huggingface_hub import login, HfApi

# Step 1: Load the Wav2Vec2 model and processor
model_id = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_id)
processor = Wav2Vec2Processor.from_pretrained(model_id)

for name,module in model.named_modules():
  print(f"{name}:{module}")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


:Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder

In [ ]:
target_modules = []
for i in range(6):
    target_modules.extend([
        f"wav2vec2.encoder.layers.{i}.attention.k_proj",
        f"wav2vec2.encoder.layers.{i}.attention.v_proj",
        f"wav2vec2.encoder.layers.{i}.attention.q_proj",
        f"wav2vec2.encoder.layers.{i}.attention.out_proj",
        f"wav2vec2.encoder.layers.{i}.feed_forward.intermediate_dense",
        f"wav2vec2.encoder.layers.{i}.feed_forward.output_dense",
    ])

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=target_modules,
)


# Step 3: Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)


In [ ]:
waveform, sample_rate = torchaudio.load("sample-1.mp3")
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# Convert stereo to mono if necessary
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0)
waveform = waveform.squeeze(0)

# Tokenize the audio input
input_values = processor(waveform, return_tensors="pt", sampling_rate=16000).input_values

# Perform STT
with torch.no_grad():
    logits = lora_model(input_values).logits

# Decode the logits to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)


In [ ]:
print(transcription[0])
def measure_inference_time(model, input_tensor):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        start_time = time.time()
        _ = model(input_tensor)  # Perform inference
        end_time = time.time()
    return end_time - start_time

# Example: Assume input_tensor is preprocessed audio tensor
input_tensor = torch.rand(1, 16000)  # Simulate an audio input of 1-second at 16kHz

# Measure for non-quantized model
inference_time_non_quant = measure_inference_time(lora_model, input_tensor)
print(f"Inference time: {inference_time_non_quant} seconds")

# Function to check RAM usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 ** 2)  # Return memory usage in MB

# Example: Measure RAM usage before and after inference for non-quantized and quantized models
print(f"Initial RAM usage: {get_memory_usage()} MB")

# Inference with non-quantized model
_ = lora_model(input_tensor)
print(f"RAM usage after quantized model: {get_memory_usage()} MB")

IT IS ELEVINDIN CASIEN NISIN NORING IS ITS MILEWARE MAY SEND A LITTLE NAY IN THE GENEW WITH RAWMEN I HAD DELDA FRAIL
Inference time: 0.4823329448699951 seconds
Initial RAM usage: 2830.98828125 MB
RAM usage after quantized model: 2830.98828125 MB
